### Loading


In [1]:
from show import *
from series import *
from colors import color
from operator import itemgetter, attrgetter

In [2]:
import pickle
completed = {}

with open('./show_data/completed_shows.pkl', 'rb') as f:
    completed = pickle.load(f)
    
with open('./show_data/series.pkl', 'rb') as f:
    series = pickle.load(f)
    
with open('./show_data/decisions.pkl', 'rb') as f:
    decisions = pickle.load(f)

### Definitions

In [3]:
def input_safe(txt):
    try:
        inp = float(input(txt))
        if inp < 0 or inp > 10:
            return "error"
        else:
            return inp
    except:
        print('Error')

def create_show(name='', tp='TV'):
    if len(name) == 0:
        name = input("Name of Show: ").strip()
    plot = float(input_safe("Story: "))
    characters = float(input_safe("Characters: "))
    art = float(input_safe("Art/Animation: "))
    music = float(input_safe("Sound: "))
    enjoy = float(input_safe("Personal Enjoyment: "))
    return Show(name, plot, characters, art, music, enjoy, tp)

def create_movie(name=''):
    return create_show(name, 'Movie')

def add_show(show):
    old = completed.pop(show.name, None)
    if old:
        show.set_type(old.get_type())
    completed[show.name] = show
    
    for title in series:
        shows = series[title].shows
        if show in shows:
            series[title].addShow(show)
            break

### Make/Update Show Entries

In [4]:
def entries(shows=[], ans=''):
    header_shown = False
    while shows or ans == 'y' or len(ans) >= 2:
        if shows:
            name = shows.pop(0)
        elif ans != 'edit':    
            name = ans
            header_shown = False

        name = name.strip()

        if len(name) > 0 :
            if name in completed:
                print(f'{color.BOLD}Current Entry:{color.END}')
                if not header_shown:
                    print(header())
                    header_shown = True
                print(str(completed[name]))
                
            try:
                x = create_show(name) # change to create_movie to add movie
                print(f'{color.BOLD}Updated Entry:{color.END}')
                print(str(x))
                add_show(x)
            except TypeError as t_e:
                print()
        else:
            if len(name) == 0:
                print('err: no name entered')
                break
        
        if not shows:
            ans = input('\nAdd another? (y/n/edit): ')
                        
ans = input('Enter name of show on MAL: ')
# ans = ''
entries(ans=ans)

KeyboardInterrupt: Interrupted by user

### Update entries by series

In [8]:
s = input('Enter name of series: ')

Enter name of series:  86


In [10]:
if s in series:
    for show in sorted(series[s].shows):
        print(show)
    print()
    entries(series[s].getShows())
    print()
    print(header())
    print(series[s])
else:
    print('Series not found')

86                                      9           8           9           10          10          8.9         9           

Current Entry:
NAME                                    STORY       CHARACTERS  VISUALS     SOUND       ENJOYMENT   RAW       OVERALL     
86                                      9           8           9           10          10          8.9         9           
Error



KeyboardInterrupt: Interrupted by user


Add another? (y/n/edit):  n
Story:  


In [11]:
sorted(completed), len(completed)

(['86',
  'Akame ga Kill!',
  'Ao no Exorcist',
  'Boku dake ga Inai Machi',
  'Boku no Hero Academia',
  'Boku no Hero Academia 2nd Season',
  'Boku no Hero Academia 3rd Season',
  'Boku no Hero Academia 4th Season',
  'Code Geass: Hangyaku no Lelouch',
  'Code Geass: Hangyaku no Lelouch R2',
  'Darling in the FranXX',
  'Death Note',
  'Enen no Shouboutai',
  'Enen no Shouboutai: Ni no Shou',
  'Fate/Zero',
  'Fate/Zero 2nd Season',
  "Fate/stay night Movie: Heaven's Feel - I. Presage Flower",
  'Fate/stay night: Unlimited Blade Works',
  'Fate/stay night: Unlimited Blade Works 2nd Season',
  'Fruits Basket 1st Season',
  'Fruits Basket 2nd Season',
  'Fruits Basket: The Final',
  'Fullmetal Alchemist: Brotherhood',
  'Given',
  'Haikyuu!!',
  'Haikyuu!! Second Season',
  'Haikyuu!!: Karasuno Koukou vs. Shiratorizawa Gakuen Koukou',
  'Haikyuu!!: To the Top',
  'Haikyuu!!: To the Top 2nd Season',
  'Hunter x Hunter (2011)',
  'Jujutsu Kaisen',
  'Kanojo, Okarishimasu',
  'Kaze ga Tsu

In [12]:
def percentMatch(show, title):
    matches = 0
    common = ''
    thresh = 8
        
    for i in range(len(show)):
        if i >= len(title):
            break
        
        if title[i] == show[i]:
            matches += 1
            common += title[i]
        else:
            break
    
#     if matches > thresh:
#         return matches, 1, common
    
    return matches, matches / max(len(show), len(title)), common.strip()

### Make Series - assisted string matching

In [50]:
# series = {}
def fmtString(num, percent, matches, title, name, common):
    return f'({str(num)}){percent*100:.2f}% match: {title} | {name} --> {common}\n'

for show in sorted(completed):
    add = False
    found = False
    
    for title in series:
        if completed[show] in series[title].shows:
            found = True
            break
    
    if not found:
        print(f'Inserting: {color.BOLD}{show}{color.END}')
        for title in series:
            matches, percent, common = percentMatch(show, title)

            if percent > 0.6 and matches > 5:
                add = True
                print(fmtString(1, percent, matches, title, show, common))
                break
            elif percent > 0.45 or matches > 5:
                if (show, title) in decisions:
                    print(fmtString(2, percent, matches, title, show, common))
                    add = decisions[(show, title)]        
                else:
                    choice = input(fmtString(3, percent, matches, title, show, common))
                    if choice == "y" or choice == "":
                        add = True

                    decisions[(show, title)] = add

                    if add:
                        break

        if add:
            shows = set()
            if common != show and common != title:
                s1 = series.pop(show, None)
                s2 = series.pop(title, None)
                if s1 is not None:
                    shows = shows.union(s1.shows)
                if s2 is not None:
                    shows = shows.union(s2.shows)

                series[common] = Series(common, shows)
            series[common].addShow(completed[show])
        else:
            series[show] = Series(show, {completed[show]})

In [51]:
sorted(series), len(series)

(['86',
  'Akame ga Kill!',
  'Ao no Exorcist',
  'Boku dake ga Inai Machi',
  'Boku no Hero Academia',
  'Code Geass: Hangyaku no Lelouch',
  'Darling in the FranXX',
  'Death Note',
  'Enen no Shouboutai',
  'Fate/Zero',
  "Fate/stay night Movie: Heaven's Feel - I. Presage Flower",
  'Fate/stay night: Unlimited Blade Works',
  'Fruits Basket',
  'Fullmetal Alchemist: Brotherhood',
  'Given',
  'Haikyuu!!',
  'Hunter x Hunter (2011)',
  'Jujutsu Kaisen',
  'Kanojo, Okarishimasu',
  'Kaze ga Tsuyoku Fuiteiru',
  'Kill la Kill',
  'Kimetsu no Yaiba',
  'Kimi no Na wa.',
  'Kiznaiver',
  'Kono Subarashii Sekai ni Shukufuku wo!',
  'Koutetsujou no Kabaneri',
  'Mahou Shoujo Madoka★Magica',
  'Nisekoi',
  'No Game No Life',
  'One Punch Man',
  'Re:Zero kara Hajimeru Isekai Seikatsu',
  'Saenai Heroine no Sodatekata',
  'Seishun Buta Yarou wa Bunny Girl Senpai no Yume wo Minai',
  'Seitokai Yakuindomo',
  'Shigatsu wa Kimi no Uso',
  'Shingeki no Bahamut: Genesis',
  'Shingeki no Kyojin',


### Saving

In [60]:
with open('./show_data/completed_shows.pkl', 'wb') as f:
    pickle.dump(completed, f)
    
with open('./show_data/series.pkl', 'wb') as f:
    pickle.dump(series, f)
    
with open('./show_data/decisions.pkl', 'wb') as f:
    pickle.dump(decisions, f)